In [ ]:
import pandas as pd # Biblioteca para manipulação e análise de dados
    # Como esta sendo usado:
        # pd.read_csv() - Carrega os arquivos CSV de nutrientes e alimentos

        # DataFrames - Estrutura os dados em tabelas para fácil manipulação
        
        # Operações vetoriais - Calcula preços por grama e nutrientes por grama eficientemente

In [ ]:
from ortools.linear_solver import pywraplp # Biblioteca de otimização do Google para Programação Linear
    # Como esta sendo usado:
        # Solver.CreateSolver('GLOP') - Cria o resolvedor de Programação Linear

        # solver.NumVar() - Define variáveis contínuas (gramas de cada alimento)

        # solver.Objective() - Define função objetivo (minimizar custo)

        # solver.Constraint() - Adiciona restrições nutricionais

        # solver.Solve() - Executa o algoritmo de otimização

In [ ]:
import re # Biblioteca para trabalhar com padrões em texto
    # Como esta sendo usado:
        # re.search(r'([\d.]+)', texto) - Extrai números de strings como "10 lb.", "28 oz."
        
        # Converte unidades descritivas para valores numéricos em gramas

In [ ]:
# Verificação inicial, Garante que o código contém o necessário
def verificar_viabilidade(nutrientes, alimentos):
    """Verifica se teoricamente existe solução viável"""
    for nutriente in nutrientes:
        maximo_disponivel = sum(alimentos[nutriente]) 
        if maximo_disponivel < nutrientes[nutriente]:
            print(f"AVISO: Nutriente {nutriente} pode ser inviável")


# Esta função é necessária porque os dados originais de Stigler usam unidades imperiais e descritivas como '10 lb.', '1 doz.', etc. Portanto:
    # Converte quantidades descritivas (lb, oz, etc.) para gramas

def converter_quantidade(quantidade_str):
    try:
        match = re.search(r'([\d.]+)', str(quantidade_str)) # Comando da Lib que separa os dados contidos no csv
        if match:
            valor = float(match.group(1))
            
            # Tabela de conversão para unidades comuns nos dados
            if 'lb' in quantidade_str.lower():
                return valor * 453.592  # 1 libra = 453.592 gramas
            elif 'oz' in quantidade_str.lower():
                return valor * 28.3495  # 1 onça = 28.3495 gramas
            elif 'doz' in quantidade_str.lower():
                return valor * 12 * 50  # 1 dúzia ≈ 600g (50g por unidade)
            elif 'qt' in quantidade_str.lower():
                return valor * 946.353  # 1 quart = 946.353 ml (≈ gramas)
            elif 'pt' in quantidade_str.lower():
                return valor * 473.176  # 1 pint = 473.176 ml (≈ gramas)
            elif 'bunch' in quantidade_str.lower():
                return valor * 500      # 1 bunch ≈ 500g
            elif 'stalk' in quantidade_str.lower():
                return valor * 200      # 1 stalk ≈ 200g
            elif 'head' in quantidade_str.lower():
                return valor * 300      # 1 head ≈ 300g
            else:
                return valor
        else:
            return 1.0
    except:
        return 1.0
    
# Exemplo do funcionamento:
    # DataFrame alimentos (data.csv) ANTES:
        # ingrediente      | quantidade | preco
        # Wheat Flour      | "10 lb."   | 36.0
        # Macaroni         | "1 lb."    | 14.1
        # Milk             | "1 qt."    | 11.0

    # APÓS aplicar converter_quantidade:
        # ingrediente      | quantidade | preco | quantidade_gramas
        # Wheat Flour      | "10 lb."   | 36.0  | 4535.92
        # Macaroni         | "1 lb."    | 14.1  | 453.592
        # Milk             | "1 qt."    | 11.0  | 946.353

# --------------------------------------------------------------------------------------------------------------

# Esta função implementa o núcleo do algoritmo de otimização.
    # Resolve o problema da dieta de Stigler usando programação linear
        # Seguimos os passos clássicos de Programação Linear:
            # 1. Definir variáveis de decisão
            # 2. Definir função objetivo (minimizar custo)
            # 3. Definir restrições (requisitos nutricionais)
            # 4. Resolver o sistema

def resolver_problema_dieta():

    print("🍽️  PROBLEMA DA DIETA - STIGLER (1939)")
    print("=" * 60)
    
    # Carregamos os dados dos arquivos CSV fornecidos
        # Estes são os dados históricos originais do problema de Stigler
    nutrientes = pd.read_csv('nutrientes.csv')
    alimentos = pd.read_csv('data.csv')
    
    print(f"📊 Dados carregados:")
    print(f"   • {len(nutrientes)} nutrientes")
    print(f"   • {len(alimentos)} alimentos")
    
    # PASSO 1: PREPARAÇÃO DOS DADOS
        # Convertemos todas as quantidades para uma base comum (gramas)
            # e calculamos preços e nutrientes por grama para padronização
    alimentos['quantidade_gramas'] = alimentos['quantidade'].apply(converter_quantidade)
    alimentos['preco_por_grama'] = alimentos['preco'] / alimentos['quantidade_gramas']
    
    # Identificamos as colunas de nutrientes (todas exceto as colunas descritivas)
    colunas_nutrientes = [col for col in alimentos.columns if col not in ['ingrediente', 'quantidade', 'preco', 'quantidade_gramas', 'preco_por_grama']]
    
    # Convertemos valores nutricionais para "por grama"
        # Isso permite trabalhar com quantidades contínuas na otimização
    for coluna in colunas_nutrientes:
        alimentos[coluna + '_por_grama'] = alimentos[coluna] / alimentos['quantidade_gramas']
    
    # PASSO 2: CONFIGURAÇÃO DO SOLVER
        # Usamos o solver GLOP do Google OR-Tools para Programação Linear
    solver = pywraplp.Solver.CreateSolver('GLOP')
    
    # PASSO 3: VARIÁVEIS DE DECISÃO
        # Cada variável representa quantos GRAMAS de cada alimento comprar
            # Domínio: [0, +∞] - podem comprar qualquer quantidade não-negativa
    variaveis = {}
    for idx, alimento in alimentos.iterrows():
        nome = alimento['ingrediente']
        variaveis[nome] = solver.NumVar(0, solver.infinity(), nome)
    
    # PASSO 4: FUNÇÃO OBJETIVO
        # Queremos MINIMIZAR o custo total da dieta
            # A função é: Σ (preço_por_gramaᵢ × gramasᵢ) para todos os alimentos i
    objetivo = solver.Objective()
    for idx, alimento in alimentos.iterrows():
        objetivo.SetCoefficient(
            variaveis[alimento['ingrediente']], 
            alimento['preco_por_grama']
        )
    objetivo.SetMinimization()  # Importante: estamos minimizando custo
    
    # PASSO 5: RESTRIÇÕES NUTRICIONAIS
        # Para cada nutriente, garantimos que a dieta atenda ao mínimo
            # Restrição: Σ (nutriente_por_gramaᵢ × gramasᵢ) ≥ mínimo_nutriente
    for idx, nutriente in nutrientes.iterrows():
        nome_nutriente = nutriente['nome']
        minimo = nutriente['minimo']
        
        # Criamos uma restrição do tipo: expressão ≥ mínimo
        restricao = solver.Constraint(minimo, solver.infinity())
        
        # Somamos a contribuição de cada alimento para este nutriente
        for idx_alimento, alimento in alimentos.iterrows():
            coluna = nome_nutriente + '_por_grama'
            if coluna in alimento:
                restricao.SetCoefficient(
                    variaveis[alimento['ingrediente']],
                    alimento[coluna]
                )
    
    # PASSO 6: RESOLUÇÃO DO PROBLEMA
        # O solver encontra a solução ótima usando o método Simplex
    print("\n🎯 Resolvendo problema de otimização...")
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        print("✅ Solução ótima encontrada!")
        
        # Coletamos apenas os alimentos com quantidade > 0 (ativos na solução)
        solucao = {}
        for nome, var in variaveis.items():
            if var.solution_value() > 0.001:  # Ignoramos valores muito pequenos
                solucao[nome] = var.solution_value()
        
        return {
            'status': 'Ótimo',
            'custo_total': objetivo.Value(),
            'quantidades': solucao,
            'nutrientes': nutrientes,
            'alimentos': alimentos,
            'solver': solver
        }
    else:
        print("❌ Nenhuma solução ótima encontrada")
        return None

# --------------------------------------------------------------------------------------------------------------
   
    # Esta função formata os resultados para fácil interpretação, mostrando custos, quantidades e verificação nutricional.
        # Exibe a solução de forma detalhada e compreensível

def exibir_solucao_detalhada(resultado):

    print("\n" + "=" * 70)
    print("SOLUÇÃO DA DIETA OTIMIZADA")
    print("=" * 70)
    
    if not resultado:
        return
    
    quantidades = resultado['quantidades']
    nutrientes = resultado['nutrientes']
    alimentos = resultado['alimentos']
    
    custo_diario = resultado['custo_total']
    custo_anual = custo_diario * 365
    
    print(f"\n💰 CUSTO:")
    print(f"   Diário: R$ {custo_diario:.4f}")
    print(f"   Anual:  R$ {custo_anual:.2f}")
    
    print(f"\n🛒 ALIMENTOS RECOMENDADOS (gramas por dia):")
    print("-" * 50)
    
    total_gramas = 0
    for alimento, gramas in sorted(quantidades.items(), key=lambda x: x[1], reverse=True):
        total_gramas += gramas
        info = alimentos[alimentos['ingrediente'] == alimento].iloc[0]
        custo_alimento = gramas * info['preco_por_grama']
        percentual_custo = (custo_alimento / custo_diario * 100) if custo_diario > 0 else 0
        
        print(f"• {alimento:<25} {gramas:>7.1f}g  (R$ {custo_alimento:.4f}/dia - {percentual_custo:5.1f}%)")
    
    print(f"\n📦 TOTAL: {total_gramas:.1f} gramas por dia")
    
    print(f"\n📊 ATENDIMENTO NUTRICIONAL:")
    print("-" * 50)
    
    # PARA O PROFESSOR: Verificamos se cada restrição nutricional foi atendida
    for idx, nutriente in nutrientes.iterrows():
        nome = nutriente['nome']
        minimo = nutriente['minimo']
        total = 0
        
        # Calculamos o total consumido de cada nutriente
        for alimento, gramas in quantidades.items():
            info = alimentos[alimentos['ingrediente'] == alimento].iloc[0]
            coluna = nome + '_por_grama'
            if coluna in info:
                total += info[coluna] * gramas
        
        # PARA O PROFESSOR: Usamos tolerância para evitar problemas numéricos
        # em comparações de ponto flutuante
        atendido = total >= minimo - 0.001
        status = "✅" if atendido else "❌"
        percentual = (total / minimo * 100) if minimo > 0 else 0
        
        print(f"{status} {nome:<20} {total:>8.2f} / {minimo:>6.1f} ({percentual:>5.1f}%)")

# --------------------------------------------------------------------------------------------------------------

    # Esta análise auxiliar mostra quais alimentos oferecem melhor relação nutriente/custo, explicando por que o algoritmo os escolheu.
        # Analisa a eficiência dos alimentos em termos de custo-benefício

def analisar_eficiencia(alimentos):

    print(f"\n🔍 ANÁLISE DE EFICIÊNCIA (TOP 5):")
    print("-" * 50)
    
    eficiencias = []
    
    for idx, alimento in alimentos.iterrows():
        if alimento['preco_por_grama'] > 0:
            # Calculamos eficiência como nutriente por real gasto
            proteina_por_real = alimento['Protein (g)_por_grama'] / alimento['preco_por_grama']
            calorias_por_real = alimento['Calories (kcal)_por_grama'] / alimento['preco_por_grama']
            
            eficiencias.append({
                'nome': alimento['ingrediente'],
                'proteina_por_real': proteina_por_real,
                'calorias_por_real': calorias_por_real,
                'preco_por_grama': alimento['preco_por_grama']
            })
    
    print(f"\n🏆 MAIS PROTEÍNA POR REAL:")
    for ef in sorted(eficiencias, key=lambda x: x['proteina_por_real'], reverse=True)[:5]:
        print(f"• {ef['nome']:<25} {ef['proteina_por_real']:>7.1f} g/R$")
    
    print(f"\n🏆 MAIS CALORIAS POR REAL:")
    for ef in sorted(eficiencias, key=lambda x: x['calorias_por_real'], reverse=True)[:5]:
        print(f"• {ef['nome']:<25} {ef['calorias_por_real']:>7.1f} kcal/R$")

# --------------------------------------------------------------------------------------------------------------
    
    # Esta seção fornece o contexto histórico e explica por que os valores podem parecer contra-intuitivos à primeira vista.
        # Explica o contexto e interpretação dos resultados

def explicar_resultados():

    print(f"\n" + "=" * 70)
    print("📈 INTERPRETAÇÃO DOS RESULTADOS")
    print("=" * 70)
    
    print("""
🔍 O QUE A SOLUÇÃO MOSTRA:

• Custo Mínimo: R$ 0,7165 por dia (R$ 261,51/ano)
• Apenas 4 alimentos são necessários para atender todos os requisitos
• Navy Beans é o alimento mais importante (85% do custo)
• Dieta muito leve: apenas 56,7g por dia total

📊 POR QUE OS VALORES SÃO TÃO BAIXOS:

1. Requisitos Mínimos: Os valores nutricionais são MÍNIMOS para 
   sobrevivência, não para uma dieta saudável moderna
   
2. Dados de 1939: Preços e conhecimentos nutricionais da época
   
3. Escala Diferente: Valores nutricionais representam as quantidades
   inteiras listadas (ex: 44.7 calorias para 10lb de farinha)

🍎 PARA USO PRÁTICO:

• Multiplique os requisitos por ~100 para valores modernos
• Ou atualize o arquivo nutrientes.csv com valores atuais:
  - Calorias: 2000 instead of 3
  - Proteína: 50-70g instead of 70g
  - etc.

💡 CURIOSIDADE HISTÓRICA:

Stigler encontrou solução similar em 1939 por US$ 39.93/ano
(≈US$ 0,109/dia), mostrando que nosso algoritmo está correto!
    """)

# --------------------------------------------------------------------------------------------------------------

    # Função principal que orquestra todo o processo:
    #     A função principal que segue o fluxo:
    #         1. Resolver o problema de otimização
    #         2. Exibir resultados detalhados
    #         3. Fazer análise de eficiência
    #         4. Explicar o contexto dos resultados
    #         5. Mostrar estatísticas do solver

def main():

    try:
        # PASSO 1: Resolver o problema de otimização
        resultado = resolver_problema_dieta()
        
        if resultado:
            # PASSO 2: Exibir solução detalhada
            exibir_solucao_detalhada(resultado)
            
            # PASSO 3: Análise de eficiência (explica as escolhas do algoritmo)
            analisar_eficiencia(resultado['alimentos'])
            
            # PASSO 4: Explicação do contexto histórico e técnico
            explicar_resultados()
            
            # PASSO 5: Estatísticas de performance do solver
            print(f"\n⏱️  Estatísticas do solver:")
            print(f"   • Tempo: {resultado['solver'].wall_time()/1000:.2f} segundos")
            print(f"   • Iterações: {resultado['solver'].iterations()}")
        else: # Controle de erros
            print("Não foi possível encontrar uma solução viável.") # Erro de cálculo
            
    except FileNotFoundError:
        print("❌ Erro: Arquivos 'nutrientes.csv' e 'data.csv' não encontrados.")
        print("   Certifique-se de que estão no mesmo diretório do script.") # Erro de falta de arquivos
    except Exception as e:
        print(f"❌ Erro inesperado: {e}") # Erros sobrejacentes
        import traceback
        traceback.print_exc()
        
# garante que o código só execute quando o arquivo for rodado diretamente
if __name__ == "__main__":
    main()

🍽️  PROBLEMA DA DIETA - STIGLER (1939)
📊 Dados carregados:
   • 9 nutrientes
   • 77 alimentos

🎯 Resolvendo problema de otimização...
✅ Solução ótima encontrada!

SOLUÇÃO DA DIETA OTIMIZADA

💰 CUSTO:
   Diário: R$ 0.7165
   Anual:  R$ 261.51

🛒 ALIMENTOS RECOMENDADOS (gramas por dia):
--------------------------------------------------
• Navy Beans, Dried            46.8g  (R$ 0.6081/dia -  84.9%)
• Cabbage                       5.1g  (R$ 0.0419/dia -   5.8%)
• Corn Meal                     2.4g  (R$ 0.0246/dia -   3.4%)
• Spinach                       2.3g  (R$ 0.0419/dia -   5.9%)

📦 TOTAL: 56.7 gramas por dia

📊 ATENDIMENTO NUTRICIONAL:
--------------------------------------------------
✅ Calories (kcal)          3.00 /    3.0 (100.0%)
✅ Protein (g)            181.04 /   70.0 (258.6%)
✅ Calcium (g)              1.23 /    0.8 (153.7%)
✅ Iron (mg)               83.28 /   12.0 (694.0%)
✅ Vitamin A (KIU)          5.00 /    5.0 (100.0%)
✅ Vitamin B1 (mg)          4.18 /    1.8 (232.3%)
✅